# 2. OpenAI の チャット API の基礎


## 2.3. 入出力の長さの制限や料金に影響する「トークン」


### トークン


In [1]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))


Chat
GPT


### Tokenizer と tiktoken の紹介


In [2]:
import tiktoken

text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))


37


## 2.5. Chat Completions API のハンズオン


### Chat Completions API の呼び出し


In [3]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
)
print(response.to_json(indent=2))


{
  "id": "chatcmpl-AlEQdq40GYhVVWngU6jnACvIV5JzL",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1735820243,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0aa8d3e20b",
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 25,
    "total_tokens": 53,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### 会話履歴を踏まえた応答を得る


In [4]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
        {
            "role": "assistant",
            "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？",
        },
        {"role": "user", "content": "私の名前が分かりますか？"},
    ],
)
print(response.to_json(indent=2))


{
  "id": "chatcmpl-AlEQehTMsGb7hMr3vy4TxJEmHWpab",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "はい、ジョンさんですね！お名前を教えていただきありがとうございます。何か特別なことについてお話ししたいことがありますか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1735820244,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0aa8d3e20b",
  "usage": {
    "completion_tokens": 33,
    "prompt_tokens": 69,
    "total_tokens": 102,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### ストリーミングで応答を得る


In [5]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)


こんにちは、ジョンさん！お会いできてうれしいです。今日はいかがですか？何かお手伝いできることがあれば教えてください！

### JSON モード


In [6]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "昔々あるところにおじいさんとおばあさんがいました",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)


{"people": ["おじいさん", "おばあさん"]}


### Vision（画像入力）


In [7]:
from openai import OpenAI

client = OpenAI()

image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)


この画像は、書籍の表紙を示しています。タイトルは「ChatGPT/LangChainによるチャットシステム構築【実践】入門」となっており、著者は吉田真吾と大嶋勇樹です。表紙には色鮮やかな鳥が描かれており、青を基調としたデザインで「ChatGPT」という文字が目立っています。内容としては、おそらくChatGPTやLangChainについての実践的なガイドを提供するもので、大規模言語モデルの活用方法について説明しているようです。


### （コラム）Completions API


In [8]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="こんにちは！私はジョンと言います！",
)
print(response.to_json(indent=2))


{
  "id": "cmpl-AlEQnTLipLIfv594cp5D207HRoRc0",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nはじめまして、ジョンさん！どんなこ"
    }
  ],
  "created": 1735820253,
  "model": "gpt-3.5-turbo-instruct",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 11,
    "total_tokens": 27
  }
}


## 2.6. Function calling


### Function calling のサンプルコード


In [9]:
import json


def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]


In [11]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=2))


{
  "id": "chatcmpl-AlEQocXzozIfpXbup7zbSqtkDTx6v",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_ko26w30xjlaFYDjaU8V8ttVV",
            "function": {
              "arguments": "{\"location\":\"東京\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1735820254,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_5f20662549",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 81,
    "total_tokens": 102,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens":

In [12]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())


In [13]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 使いたい関数は複数あるかもしれないのでループ
for tool_call in response_message.tool_calls:
    # 関数を実行
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 関数の実行結果を会話履歴としてmessagesに追加
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )


{"location": "\u6771\u4eac", "temperature": "unknown"}


In [14]:
print(json.dumps(messages, ensure_ascii=False, indent=2))


[
  {
    "role": "user",
    "content": "東京の天気はどうですか？"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_ko26w30xjlaFYDjaU8V8ttVV",
        "function": {
          "arguments": "{\"location\":\"東京\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_ko26w30xjlaFYDjaU8V8ttVV",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"\\u6771\\u4eac\", \"temperature\": \"unknown\"}"
  }
]


In [15]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))


{
  "id": "chatcmpl-AlEQpXrSXSpe2ZtTwKnifM5JNHJXX",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "申し訳ありませんが、現在の東京の具体的な天気情報を取得することができませんでした。最新の天気情報を知りたい場合は、信頼できる天気予報のウェブサイトやアプリをご利用ください。",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1735820255,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_5f20662549",
  "usage": {
    "completion_tokens": 56,
    "prompt_tokens": 63,
    "total_tokens": 119,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}
